In [11]:
import os
import time

# from dotenv import load_dotenv
import requests
import json
import re
from datetime import date
import csv
import pandas as pd

from datetime import datetime
import pytz


#load_dotenv()

API_KEY="bc7791c727dae92fcf26b6e1338575e9"
HISTORICAL_API_KEY="668b5200589eb79da855b2f68cd0749a"
BASE = "https://api.the-odds-api.com/v4"


In [56]:
class Collector:

    sports_titles = []
    sports_keys = []

    @staticmethod
    def getAPIKey():
        print(HISTORICAL_API_KEY)


    @staticmethod
    def getEvents(sport):
        url = f'{BASE}/sports/{sport}/events?apiKey={API_KEY}'

        response = requests.request("GET", url, headers={}, data={})

        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Error Collecting Events Data. Status code: {response.status_code}")
            data = None

        if data is not None:
            with open(f"{sport}-event-{date.today()}.json", "w") as json_file:
                json.dump(data, json_file, indent=4)

    @staticmethod
    def getHistoricalEvents(sport, date):

        # ISO format: 2021-10-18T12:00:00Z
        # yyyy mm dd

        if not Collector.validateDate(date):
            print(f"Invalid date: {date}")
            return

        orig_date = date
        date += "T10:00:00Z"  # auto collect at 10 AM

        url = f"{BASE}/historical/sports/{sport}/events?apiKey={HISTORICAL_API_KEY}&date={date}"

        print(url)

        response = requests.request("GET", url, headers={}, data={})

        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Error Collecting Events Data. Status code: {response.status_code}")
            data = None
            return None

        # print(data)

        if data is not None:
            with open(f"basketball-events/{sport}-event-{orig_date}.json", "w") as json_file:
                json.dump(data, json_file, indent=4)

        return "working"

    @staticmethod
    def getPropByEventFiles(dir, sport, markets, regions):
        # gets all props for every single event in basketball-events
        files_collected = []
        events_found = 0

        for file in os.listdir(dir):
            if os.path.isfile(os.path.join(dir, file)) and file.endswith('.json'):
                with open(os.path.join(dir, file), "r") as f:
                    data = json.load(f)

                timestamp = data["timestamp"]
                est_file_date = Collector.convertISOtoEST(timestamp)

                files_collected.append(os.path.join(dir, file))
                for event in data["data"]:
                    eventId = event["id"]

                    est_start_date = Collector.convertISOtoEST(event["commence_time"])


                    if est_start_date == est_file_date:
                        events_found += 1
                        d = Collector.getHistoricalNBAPropsByEventId(sport, eventId, markets, regions, timestamp, False, True)
                        print(eventId)
                        if d is None:
                            print(files_collected)
                            return
                        time.sleep(.51)


        print(events_found)
        print(files_collected)

    @staticmethod
    def format_date(date_str):
        parts = date_str.split('-')
        if len(parts) == 3:
            year = parts[0]
            month = parts[1].zfill(2)  # Zero-padding month
            day = parts[2].zfill(2)    # Zero-padding day
            format_date = f"{year}-{month}-{day}"

            if format_date == "2024-01-00":
                return "2023-12-31"
            else:
                return format_date
        else:
            return None  # Invalid date format

    @staticmethod
    def convertISOtoEST(iso_time):
        # Split the ISO 8601 string to extract date and time components
        date_part, time_part = iso_time.split('T')

        # Extract hour, minute, and second components from the time part
        hour, minute, second = map(int, time_part[:-1].split(':'))  # Remove the 'Z' at the end

        # Adjust the hour to EST timezone (UTC-5)
        hour -= 5

        # If the adjusted hour is negative, roll back to the previous day
        if hour < 0:
            hour += 24
            # Adjust the date part accordingly
            year, month, day = map(int, date_part.split('-'))
            previous_day = (year, month, day - 1)
            date_part = '-'.join(map(str, previous_day))

        # Format the date part
        date_part = Collector.format_date(date_part)

        # Construct the EST time string
        est_time = f"{date_part}"

        return est_time

    @staticmethod
    def convertPropFilesToSingularCSV(dir):
        csv_player_data = []

        empty_files = []

        for file in os.listdir(dir):
            if os.path.isfile(os.path.join(dir, file)) and file.endswith('.json'):
                with open(os.path.join(dir, file), "r") as f:
                    data = json.load(f)

                filename = os.path.join(dir, file)
                date = filename.split("props-")[1].replace(".json", "")


                data = data["data"]

                player_data = {}

                if (len(data["bookmakers"]) == 0):
                    empty_files.append(filename)
                else:
                    for market in data["bookmakers"][0]["markets"]:
                        key = market["key"]

                        for odds in market["outcomes"]:
                            player = odds["description"]
                            if player not in player_data:
                                player_data[player] = {}

                            if key not in player_data[player]:
                                player_data[player][key] = {}

                            player_data[player][key]["line"] = odds["point"]
                            player_data[player][key][odds["name"].lower()] = odds["price"]


                    for player, lines in player_data.items():
                        row = {'Player': player, 'Date': date}
                        for prop_type, values in lines.items():
                            for prop_info, value in values.items():
                                row[f"{prop_type}_{prop_info}"] = value
                        csv_player_data.append(row)


        headers = set()
        for row in csv_player_data:
            headers.update(row.keys())

        # Write CSV data to file
        with open('odds_output.csv', 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=sorted(headers))
            writer.writeheader()
            writer.writerows(csv_player_data)

        print(empty_files)


    @staticmethod
    def getHistoricalNBAPropsByEventId(sport, eventId, markets, regions, date, testing, timestampUsed):
        if testing:
            with open("basketball-player-props/da359da99aa27e97d38f2df709343998-props-2023-11-29.json", "r") as file:
                d = json.load(file)
                # print(d)
                return

        # ISO format: 2021-10-18T12:00:00Z
        # yyyy mm dd

        if not timestampUsed and not Collector.validateDate(date):
            print(f"Invalid date: {date}")
            return

        if not timestampUsed:
            orig_date = date
            date += "T10:00:00Z" # auto collect at 10 AM
        else:
            orig_date = date.split("T")[0]

        # /v4/historical/sports/{sport}/events/{eventId}/odds?apiKey={apiKey}&regions={regions}&markets={markets}&dateFormat={dateFormat}&oddsFormat={oddsFormat}&date={date}
        url = f'{BASE}/historical/sports/{sport}/events/{eventId}/odds?apiKey={HISTORICAL_API_KEY}&regions={",".join(regions)}&markets={",".join(markets)}&date={date}'

        # print(url)
        if not testing:
            response = requests.request("GET", url, headers={}, data={})
        #print(response.json())


        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Error Collecting Player Props Data for {eventId}. Status code: {response.status_code}")
            data = None

            return None


        est_date = Collector.convertISOtoEST(date)

        if data is not None:
            with open(f"basketball-player-props/{eventId}-props-{est_date}.json", "w") as json_file:
                json.dump(data, json_file, indent=4)

        return data



    @staticmethod
    def getNBAPropsByEventId(sport, eventId, markets, regions):
        url = f'{BASE}/sports/{sport}/events/{eventId}/odds?markets={",".join(markets)}&regions={",".join(regions)}&apiKey={API_KEY}'

        response = requests.request("GET", url, headers={}, data={})

        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Error Collecting Player Props Data for {eventId}. Status code: {response.status_code}")
            data = None

        if data is not None:
            with open(f"{eventId}-props-{date.today()}.json", "w") as json_file:
                json.dump(data, json_file, indent=4)


    @staticmethod
    def exploreHistoricalPropData(filename):
        date = filename.split("props-")[1].replace(".json", "")

        with open(filename) as f:
            data = json.load(f)

        data = data["data"]

        player_data = {}

        for market in data["bookmakers"][0]["markets"]:
            key = market["key"]

            for odds in market["outcomes"]:
                player = odds["description"]
                if player not in player_data:
                    player_data[player] = {}

                if key not in player_data[player]:
                    player_data[player][key] = {}

                player_data[player][key]["line"] = odds["point"]
                player_data[player][key][odds["name"].lower()] = odds["price"]


        csv_player_data = []
        for player, lines in player_data.items():
            row = {'Player': player, 'Date': date}
            for prop_type, values in lines.items():
                for prop_info, value in values.items():
                    row[f"{prop_type}_{prop_info}"] = value
            csv_player_data.append(row)


        headers = set()
        for row in csv_player_data:
            headers.update(row.keys())

        # Write CSV data to file
        with open('output.csv', 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=sorted(headers))
            writer.writeheader()
            writer.writerows(csv_player_data)


    @staticmethod
    def validateRegion(reg):
        regex_pattern = r'^(us|us2|uk|au|eu)(,(us|us2|uk|au|eu))*$'
        return bool(re.match(regex_pattern, reg))

    @staticmethod
    def validateMarket(market):
        regex_pattern = r'^(h2h|spreads|totals|outrights|h2h_lay|outrights_lay)(,(h2h|spreads|totals|outrights|h2h_lay|outrights_lay))*$'
        return bool(re.match(regex_pattern, market))

    @staticmethod
    def validateDate(date):
        regex_pattern = r'^\d{4}-\d{2}-\d{2}$'
        return bool(re.match(regex_pattern, date))

Get all NBA events

In [42]:
# Code checks how many unique game dates there are

df = pd.read_csv('COMPLETE_player_data.csv')

unique_dates = df['Date'].value_counts()

unique_dates = unique_dates.sort_index()

filtered_dates = []

for date, count in unique_dates.items():
    spdate = date.split("-")
    if (spdate[1] == "12" and int(spdate[2]) > 12) or (spdate[1] == "01") :
        filtered_dates.append((date, count))
print(filtered_dates)


[('2023-12-13', 105), ('2023-12-14', 71), ('2023-12-15', 90), ('2023-12-16', 105), ('2023-12-17', 45), ('2023-12-18', 128), ('2023-12-19', 37), ('2023-12-20', 116), ('2023-12-21', 81), ('2023-12-22', 69), ('2023-12-23', 137), ('2023-12-25', 46), ('2023-12-26', 89), ('2023-12-27', 60), ('2023-12-28', 74), ('2023-12-29', 101), ('2023-12-30', 59), ('2023-12-31', 57), ('2024-01-01', 81), ('2024-01-02', 56), ('2024-01-03', 124), ('2024-01-04', 18), ('2024-01-05', 140), ('2024-01-06', 43), ('2024-01-07', 89), ('2024-01-08', 58), ('2024-01-09', 51), ('2024-01-10', 103), ('2024-01-11', 51), ('2024-01-12', 94), ('2024-01-13', 69), ('2024-01-14', 48), ('2024-01-15', 111), ('2024-01-16', 29), ('2024-01-17', 91), ('2024-01-18', 54), ('2024-01-19', 78), ('2024-01-20', 89), ('2024-01-21', 64), ('2024-01-22', 85), ('2024-01-23', 60), ('2024-01-24', 73), ('2024-01-25', 82), ('2024-01-26', 81), ('2024-01-27', 105), ('2024-01-28', 56), ('2024-01-29', 120), ('2024-01-30', 55), ('2024-01-31', 101)]


In [46]:
Collector.getHistoricalEvents("basketball_nba", filtered_dates[0][0])

https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-13T10:00:00Z
{'timestamp': '2023-12-13T09:55:38Z', 'previous_timestamp': '2023-12-13T09:50:38Z', 'next_timestamp': '2023-12-13T10:00:39Z', 'data': [{'id': 'ce6c9aa55ee8613be3168bf5031920bf', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-12-14T00:10:00Z', 'home_team': 'Detroit Pistons', 'away_team': 'Philadelphia 76ers'}, {'id': '532e7f4618fc67f69e19b99394ed0d06', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-12-14T00:10:00Z', 'home_team': 'Washington Wizards', 'away_team': 'New Orleans Pelicans'}, {'id': '62b3bd979a162d755689d1f073281ba4', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-12-14T00:40:00Z', 'home_team': 'Toronto Raptors', 'away_team': 'Atlanta Hawks'}, {'id': 'd73b5a32c4fed2c803e8a0b60cb8cfaa', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time':

In [49]:
sport = "basketball_nba"

In [52]:
import time

In [54]:
print(len(filtered_dates))

49


In [55]:
for i in range(5, len(filtered_dates)):
    print(i)
    resp = Collector.getHistoricalEvents(sport, filtered_dates[i][0])
    if resp is None:
        break
    time.sleep(1.01)

5
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-18T10:00:00Z
6
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-19T10:00:00Z
7
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-20T10:00:00Z
8
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-21T10:00:00Z
9
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-22T10:00:00Z
10
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-23T10:00:00Z
11
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-25T10:00:00Z
12
https://api.the

check how many games will need to collect for

In [54]:
dir = "basketball-events"

games_count = 0

games_by_day = []
for file in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, file)) and file.endswith('.json'):
        with open(os.path.join(dir, file), "r") as f:
            data = json.load(f)

        est_timestamp = Collector.convertISOtoEST(data["timestamp"])

        games_in_file = 0
        for d in data["data"]:
            start_time = Collector.convertISOtoEST(d["commence_time"])
            if est_timestamp == start_time:
                games_in_file+=1

        games_by_day.append((est_timestamp, games_in_file))

        games_count += games_in_file

print(games_count)

291


In [92]:
# Clean all timestamp and commence time mismatches

dir = "basketball-player-props"

bad_files = []

for file in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, file)) and file.endswith('.json'):
        with open(os.path.join(dir, file), "r") as f:
            data = json.load(f)
        if data["timestamp"].split("T")[0] != data["data"]["commence_time"].split("T")[0]:
            bad_files.append(os.path.join(dir, file))

print(bad_files)

['basketball-player-props/a15de2fac0aea8e1e652462401125c8d-props-2023-12-21.json', 'basketball-player-props/22eab68578d5078c14837e5f8d6f2b3c-props-2023-12-20.json', 'basketball-player-props/75a5579bbe0e162efb7b76a5150114dc-props-2023-12-19.json', 'basketball-player-props/f3189d21c6f15e3be26715e333b6a62c-props-2023-12-19.json', 'basketball-player-props/ca81cc197a2d25deafb7e70e1ff5db27-props-2023-12-19.json', 'basketball-player-props/96d0c38f9eae8242f2e363bdedd90c53-props-2023-12-20.json', 'basketball-player-props/184da460295c1cfd98b0ff31e8fc2b13-props-2023-12-21.json', 'basketball-player-props/184da460295c1cfd98b0ff31e8fc2b13-props-2023-12-20.json', 'basketball-player-props/9cb979b834e23b9dc8737eebba4db760-props-2023-12-19.json', 'basketball-player-props/20fc47de6083f99da2027876dfebadb7-props-2023-12-19.json', 'basketball-player-props/5a4b28938902c4fba16da19c6e8c1296-props-2023-12-19.json', 'basketball-player-props/96d0c38f9eae8242f2e363bdedd90c53-props-2023-12-21.json', 'basketball-pla

In [93]:
for bad_file in bad_files:
    print(bad_file)
    os.remove(bad_file)

basketball-player-props/a15de2fac0aea8e1e652462401125c8d-props-2023-12-21.json
basketball-player-props/22eab68578d5078c14837e5f8d6f2b3c-props-2023-12-20.json
basketball-player-props/75a5579bbe0e162efb7b76a5150114dc-props-2023-12-19.json
basketball-player-props/f3189d21c6f15e3be26715e333b6a62c-props-2023-12-19.json
basketball-player-props/ca81cc197a2d25deafb7e70e1ff5db27-props-2023-12-19.json
basketball-player-props/96d0c38f9eae8242f2e363bdedd90c53-props-2023-12-20.json
basketball-player-props/184da460295c1cfd98b0ff31e8fc2b13-props-2023-12-21.json
basketball-player-props/184da460295c1cfd98b0ff31e8fc2b13-props-2023-12-20.json
basketball-player-props/9cb979b834e23b9dc8737eebba4db760-props-2023-12-19.json
basketball-player-props/20fc47de6083f99da2027876dfebadb7-props-2023-12-19.json
basketball-player-props/5a4b28938902c4fba16da19c6e8c1296-props-2023-12-19.json
basketball-player-props/96d0c38f9eae8242f2e363bdedd90c53-props-2023-12-21.json
basketball-player-props/8b4d2b3361cdcda402582dfdb9e5

In [18]:
# Example inputs
iso_input_1 = "2023-12-21T00:10:00Z"
iso_input_2 = "2023-12-20T09:55:38Z"

print(Collector.convertISOtoEST(iso_input_1))
print(Collector.convertISOtoEST(iso_input_2))


2023-12-20
2023-12-20


In [57]:
#Collector.getPropByEventFiles("basketball-events-december", "basketball_nba", ["player_points", "player_rebounds", "player_assists"], ["us"])

8811ddb0d366db1fd65c124566629813
31086b15e5261feb16dbdc162c179d5b
332383bf7215581786c9cbfd78ba1795
56dd286c7b687a83f42845b574d52fe3
361d1c3e2282ef4c858c10568deaa7eb
467b39a8b732f8cd592bbe840c34250a
b13dc2d9f54ba07ac49da52f10b508eb
5808d941d68c0ed447581ddb43da48eb
1ae80cb4d2a2f36dd4af929412f5555b
c5557110bef1e8e4ad64ff9e9fa867e2
fc02d1a39b53c20230af13d9b5dd15d1
41dfd7e61e0b40fc0f10d98fa473cbef
864b43b398b818e59f51a556ee596044
5642e6bedfa5b46e68ee9ea9ebb7c39d
b0dc85ec5e468aca414d3ee313b88186
4869ddd191401fe115d3b2ab76465611
7498e9ec73fcacfcbdf85c985e7f1cab
13329da1fbc39b449e6836142193640c
6091c5c29e4cb508bca51e72bb2f6c87
eb434b59d540e88c93f543ec92975b7a
25348b429f9584571659f556664423a6
0acb786cec1a76fe87811b4b42b4061b
c55ed9921bfce15d20afabb16e7c9069
6dae88a707aada220458191e794710e3
b59d6a3fdfcc30b542c178f27b6496c9
57ed0b1714d3d5e6a4c90b46edfa6fca
a2b8a30ff6bd548cd8e7580643099746
0a25fa87e6a970c44bd5770e4b54440f
291781e08309934322de7153e4d6a25a
1126fa50abd910e80868523920c4516d
633814cf1e

In [58]:
#Collector.convertPropFilesToSingularCSV("basketball-player-props")

['basketball-player-props/f3bac99f387d25a715ea5f00b5f0fc76-props-2023-12-21.json', 'basketball-player-props/278637ae0d466211ddc5c15f097cb5ee-props-2023-12-29.json', 'basketball-player-props/7f3996cc5976eb307b77e036081da9fb-props-2023-12-23.json', 'basketball-player-props/05f41c702f32852542143ee87787049b-props-2023-12-29.json', 'basketball-player-props/b13dc2d9f54ba07ac49da52f10b508eb-props-2023-12-26.json', 'basketball-player-props/174186f9d7a4b087e44911a23abc8641-props-2023-12-29.json']
